## Scikit-Learn
Scikit-learn es una librería de Machine Learning de código abierto que contiene herramientas para pre-procesamiento de datos, entrenamiento y generación de predicciones con diferentes modelos clásicos y selección y validación de modelos, entre otras.

### 1. Flujo de trabajo en convencional en Scikit-Learn
En esencia:

- Podemos implementar modelos para tareas de aprendizaje supervisado (como clasificación o regresión) o para aprendizaje NO supervisado (como *clustering*)
- En el **pre-procesamiento** generalmente debemos **generar particiones** de los sets de datos (en entrenamiento, validación y prueba) o realizar **transformaciones** de los datos (como el escalamiento)
- Luego **creamos una instancia del algoritmo**, **entrenamos** el modelo y lo **validamos** con los sets de entrenamiento, validación y/o prueba
- Finalmente, el modelo está listo para **generar predicciones**

Veamos de forma práctica los elementos básicos de cada una de estas etapas.

### 2. Pre-procesamiento: generar particiones
Una tarea común consiste en dividir el set de datos en los sets de entrenamiento, validación y prueba.

Esto lo podemos hacer con la función train_test_split.

Para entender cómo usarla comencemos leyendo el set de datos particiones_datos_balanceados.npz (arreglo de NumPy):

In [1]:
import numpy as np 
# Datos balanceados 
X = np.load("Data/particiones-datos-balanceados.npz")['X']
Y = np.load('Data/particiones-datos-balanceados.npz')['Y']

print(X.shape)
print(Y.shape)

(20, 3)
(20,)


In [2]:
X # Datos en X 

array([[-0.75275929,  1.11852895, -7.5592353 ],
       [ 2.70428584, -3.60506139, -0.0964618 ],
       [ 1.39196365, -2.07855351, -9.31222958],
       [ 0.59195091, -1.33638157,  8.18640804],
       [-2.06388816, -0.43930016, -4.82440037],
       [-2.06403288,  2.85175961,  3.25044569],
       [-2.65149833, -3.00326218, -3.76577848],
       [ 2.19705687,  0.14234438,  0.40136042],
       [ 0.60669007,  0.92414569,  0.93420559],
       [ 1.24843547, -4.53549587, -6.30291089],
       [-2.87649303,  1.07544852,  9.39169256],
       [ 2.81945911, -3.29475876,  5.50265647],
       [ 1.99465584, -4.34948407,  8.78997883],
       [-1.72596534,  4.48885537,  7.89654701],
       [-1.9090502 ,  4.65632033,  1.95799958],
       [-1.89957294,  3.08397348,  8.4374847 ],
       [-1.17454654, -1.95386231, -8.23014996],
       [ 0.14853859, -4.02327886, -6.08034275],
       [-0.40832989,  1.84233027, -9.09545422],
       [-1.25262516, -0.59847506, -3.49339338]])

In [3]:
Y  # Datos en Y 

array([1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
      dtype=int64)

En este caso tenemos un set de datos supervisado, con:

- `X`: el arreglo de entrada al modelo (20 datos x 3 características)
- `Y`: la variable que deberá aprender a predecir el modelo (20 datos)

> **Nota importante:** los arreglos siempre deben estar dimensionados como *n_datos x n_características* (X) y *n_datos* (Y)

Supongamos que queremos realizar la partición con estas proporciones:

- Entrenamiento: 60%
- Validación: 20%
- Prueba: 20%

En este caso debemos usar `train_test_split` dos veces:

- En el primer paso partimos el set de datos en 2: 60% (entrenamiento) y 40% (resto)
- En el segundo paso partimos el set de datos restante en 2 mitades: 50% (20% del dataset original, validación) y 50% (20% del dataset original, prueba)

Veamos cómo implementar esta partición:

In [4]:
from sklearn.model_selection import train_test_split

# Particion en 60% Train y resto 40% 
x_train, x_resto, y_train, y_resto = train_test_split(X, Y, test_size=0.4, random_state=123)

# Particion "resto en 2 mitades" en 20% en validacion y 20% prueba 
x_val, x_test, y_val, y_test = train_test_split(x_resto, y_resto, test_size=0.5, random_state=321) 
# Test_size sirve para conocer en como vamos a partir los datos 0.5 es 50%; 0.4 es 60% (train) y 40% (test) 
# Random_state es semilla de generador aleatorio 

# Verificamos 
print("Tamaños")
print("\tDataset original: ", X.shape, Y.shape)
print('\tEntrenamiento: ', x_train.shape, y_train.shape)
print('\tValidación: ', x_val.shape, y_val.shape)
print('\tPrueba: ', x_test.shape, y_test.shape)

print('Proporciones categorías (0s/1s): ')
print(f'\tDataset original: {np.sum(Y==0)/len(Y)}/{np.sum(Y==1)/len(Y)}')
print(f'\tEntrenamiento: {np.sum(y_train==0)/len(y_train)}/{np.sum(y_train==1)/len(y_train)}')
print(f'\tValidación: {np.sum(y_val==0)/len(y_val)}/{np.sum(y_val==1)/len(y_val)}')
print(f'\tPrueba: {np.sum(y_test==0)/len(y_test)}/{np.sum(y_test==1)/len(y_test)}')


Tamaños
	Dataset original:  (20, 3) (20,)
	Entrenamiento:  (12, 3) (12,)
	Validación:  (4, 3) (4,)
	Prueba:  (4, 3) (4,)
Proporciones categorías (0s/1s): 
	Dataset original: 0.55/0.45
	Entrenamiento: 0.5833333333333334/0.4166666666666667
	Validación: 0.5/0.5
	Prueba: 0.5/0.5


En el caso anterior lo que hace `train_test_split` es:

1. Mezclar aleatoriamente el set de datos
2. Generar las particiones con las proporciones correspondientes

Es decir, en últimas crea cada subset usando **muestreo aleatorio**.

Este muestreo aleatorio es adecuado si por ejemplo estamos implementando un clasificador y las categorías están balanceadas, es decir, tienen más o menos la misma proporción de una categoría o de otra (como es el caso del ejemplo anterior).

Sin embargo, este muestreo aleatorio no es adecuado si tenemos datos desbalanceados, es decir con una proporción mayor de una categoría que de otra.

Por ejemplo, leamos el dataset `particiones-datos-desbalanceados.npz` y veamos la proporción de los datos:

In [6]:
# Leer datos desbalanceados
X = np.load('Data/particiones-datos-desbalanceados.npz')['X']
Y = np.load('Data/particiones-datos-desbalanceados.npz')['Y']

print(X)
print(Y)

[[-0.75275929  1.11852895 -7.5592353 ]
 [ 2.70428584 -3.60506139 -0.0964618 ]
 [ 1.39196365 -2.07855351 -9.31222958]
 [ 0.59195091 -1.33638157  8.18640804]
 [-2.06388816 -0.43930016 -4.82440037]
 [-2.06403288  2.85175961  3.25044569]
 [-2.65149833 -3.00326218 -3.76577848]
 [ 2.19705687  0.14234438  0.40136042]
 [ 0.60669007  0.92414569  0.93420559]
 [ 1.24843547 -4.53549587 -6.30291089]
 [-2.87649303  1.07544852  9.39169256]
 [ 2.81945911 -3.29475876  5.50265647]
 [ 1.99465584 -4.34948407  8.78997883]
 [-1.72596534  4.48885537  7.89654701]
 [-1.9090502   4.65632033  1.95799958]
 [-1.89957294  3.08397348  8.4374847 ]
 [-1.17454654 -1.95386231 -8.23014996]
 [ 0.14853859 -4.02327886 -6.08034275]
 [-0.40832989  1.84233027 -9.09545422]
 [-1.25262516 -0.59847506 -3.49339338]]
[1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]


In [7]:
print(X.shape)
print(Y.shape)


(20, 3)
(20,)


In [8]:
# Proporción categorías 1 y 0
print('Proporciones categorías (0s/1s) set desbalanceado: ')
print(f'\t{np.sum(Y==0)/len(Y)}/{np.sum(Y==1)/len(Y)}')

Proporciones categorías (0s/1s) set desbalanceado: 
	0.8/0.2


Claramente es un set desbalanceado: 85% categoría "0" y 15% categoría "1".

Así que si hacemos la partición y queremos por ejemplo implementar un modelo de detección de anomalías **debemos preservar estas proporciones**.

Esto se logra usando `train_test_split` pero usando un **muestreo estratificado**. En este caso simplemente usamos el argumento `stratify = Y` para que al hacer el muestreo la función tenga en cuenta las proporciones presentes en el arreglo `Y`:

In [12]:
# Particion de con muestreo estratificado 
# Particion 60% (train) y resto (40%)
x_train, x_resto, y_train, y_resto = train_test_split(X, Y, test_size=0.4, random_state=20, stratify= Y)
#*** MUESTREO ESTRATIFICADO
# particion "resto" en 2 mitades (Tambien estratficando)
x_val, x_test, y_val, y_test = train_test_split(x_resto, y_resto, test_size= 0.5, random_state=321, stratify= y_resto)
# *** MUESTREO ESTRATIFICADO  

# Verificación
print('Tamaños: ')
print('\tDataset original: ', X.shape, Y.shape)
print('\tEntrenamiento: ', x_train.shape, y_train.shape)
print('\tValidación: ', x_val.shape, y_val.shape)
print('\tPrueba: ', x_test.shape, y_test.shape)

print('Proporciones categorías (0s/1s): ')
print(f'\tDataset original: {np.sum(Y==0)/len(Y)}/{np.sum(Y==1)/len(Y)}')
print(f'\tEntrenamiento: {np.sum(y_train==0)/len(y_train)}/{np.sum(y_train==1)/len(y_train)}')
print(f'\tValidación: {np.sum(y_val==0)/len(y_val)}/{np.sum(y_val==1)/len(y_val)}')
print(f'\tPrueba: {np.sum(y_test==0)/len(y_test)}/{np.sum(y_test==1)/len(y_test)}')


Tamaños: 
	Dataset original:  (20, 3) (20,)
	Entrenamiento:  (12, 3) (12,)
	Validación:  (4, 3) (4,)
	Prueba:  (4, 3) (4,)
Proporciones categorías (0s/1s): 
	Dataset original: 0.8/0.2
	Entrenamiento: 0.8333333333333334/0.16666666666666666
	Validación: 0.75/0.25
	Prueba: 0.75/0.25


### 3. Pre-procesamiento: transformadores

> Permiten transformar los datos: escalar (`RobustScaler`, `MinMaxScaler`, `StandardScaler`), codificar (`LabelEncoder`, `OneHotEncoder`) o reducir (`PCA`), entre otras

Los pasos para usar un transformador son:

1. Crear una instancia del transformador
2. Usar el método `fit_transform()` para transformar el set de entrenamiento
3. Usar el método `transform()` para transformar los sets de validación y prueba


Por ejemplo, veamos los rangos de valores de cada columna en los sets de entrenamiento, validación y prueba (`x_train`, `x_val` y `x_test`):

In [13]:
print(f'x_train: {x_train.min(axis=0)}/{x_train.max(axis=0)}')
print(f'x_val: {x_val.min(axis=0)}/{x_val.max(axis=0)}')
print(f'x_test: {x_test.min(axis=0)}/{x_test.max(axis=0)}')

x_train: [-2.87649303 -4.34948407 -9.31222958]/[2.81945911 3.08397348 9.39169256]
x_val: [-2.65149833 -3.00326218 -9.09545422]/[2.19705687 1.84233027 0.40136042]
x_test: [-1.9090502  -4.53549587 -8.23014996]/[1.24843547 4.65632033 7.89654701]


Vemos que las variables (columnas) tienen diferentes rangos: -3 a 3, -4 a 5 y -9 a 9 aproximadamente.

Así que un tipo de pre-procesamiento sería, por ejemplo, escalar cada columna al mismo rango de valores antes de llevar los datos al modelo.

Por ejemplo, supongamos que haremos el escalamiento en el rango de -1 a 1 para lo cual podemos usar el transformador `MinMaxScaler`.

Veamos cada uno de los pasos a llevar a cabo. En primer lugar creamos una instancia del transformador:

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))

El segundo paso es usar el método `fit_transform()` aplicado sobre el set de entrenamiento (`x_train`). Este método:

- Calculará y almacenará en la instancia los mínimos y máximos de cada columna de `x_train`
- Y luego escalará `x_train` al rango de -1 a 1 usando los máximos y mínimos recién calculados

Veamos este segundo paso:

In [15]:
# fit_transform() sobre el set de entrenamiento
x_train_s = scaler.fit_transform(x_train)

In [16]:
print(f'Mínimos de "x_train": {x_train.min(axis=0)}')
print(f'Mínimos calculados por el escalador: {scaler.data_min_}')
print('-'*50)
print(f'Máximos de "x_train": {x_train.max(axis=0)}')
print(f'Máximos calculados por el escalador: {scaler.data_max_}')

Mínimos de "x_train": [-2.87649303 -4.34948407 -9.31222958]
Mínimos calculados por el escalador: [-2.87649303 -4.34948407 -9.31222958]
--------------------------------------------------
Máximos de "x_train": [2.81945911 3.08397348 9.39169256]
Máximos calculados por el escalador: [2.81945911 3.08397348 9.39169256]


Y verifiquemos que `x_train_s` contiene ahora los datos escalados al rango de -1 a 1:

In [17]:
print(f'x_train_s: {x_train_s.min(axis=0)}/{x_train_s.max(axis=0)}')

x_train_s: [-1. -1. -1.]/[1. 1. 1.]


El tercer paso es tomar el escalador (scaler) y usar el método transform() para transformar (escalar) los sets de validación (x_val) y prueba (x_test):

In [18]:
x_val_s = scaler.transform(x_val)
x_test_s = scaler.transform(x_test)

Verifiquemos que ahora el rango de valores en estos dos sets está entre -1 y 1:

In [19]:
print(f'x_val_s: {x_val_s.min(axis=0)}/{x_val_s.max(axis=0)}')
print(f'x_test_: {x_test_s.min(axis=0)}/{x_test_s.max(axis=0)}')

x_val_s: [-0.92099839 -0.63779388 -0.97682033]/[0.78145805 0.66593117 0.03866878]
x_test_: [-0.66030514 -1.05004718 -0.88429383]/[0.44837189 1.42304589 0.84012492]


Vemos que se realiza el escalamiento pero los valores mínimos y máximos no son exactamente -1 y 1. Esto debido a que el escalamiento se realiza **con base en los valores máximos y mínimos del set de entrenamiento** que no necesariamente son iguales a los de los sets de validación y prueba.

### 4. Crear, entrenar y validar el modelo: estimadores

En Scikit-Learn los modelos se denominan estimadores.

La secuencia de uso es la siguiente:

1. Crear una instancia del estimador
2. Entrenar el modelo con el set de entrenamiento y el método `fit()`
3. Validar el modelo con los sets de entrenamiento y validación usando el método `score()`
4. Poner a prueba el modelo con el set de prueba y usando los métodos `predict()` y `score()`

Veamos en detalle cada uno de estos pasos. Supongamos que tomaremos el set de datos que hemos venido usando para crear, entrenar, validar y poner a prueba un Bosque Aleatorio.

El primer paso es crear una instancia de este estimador (`RandomForestClassifier`):

In [20]:
from sklearn.ensemble import RandomForestClassifier

# Instancia 
bosque = RandomForestClassifier()

El segundo paso es entrenarlo. Para ello usamos el método fit() y le presentamos como argumentos el set de entrenamiento (x_train, y_train):

In [21]:
# 2. Entrenamiento
bosque.fit(x_train, y_train)

RandomForestClassifier()

El tercer paso es validar el modelo. Esto quiere decir que la idea es comparar el desempeño con los sets de entrenamiento y validación, para determinar si hay o no *overfitting* u *underfitting*.

El desempeño es simplemente una métrica que cuantifica qué tan bien lo esta haciendo el modelo.

Verifiquemos en este caso cuál es el desempeño usado por defecto por el bosque aleatorio:

In [22]:
bosque.score

<bound method ClassifierMixin.score of RandomForestClassifier()>

Vemos que el desempeño se está midiendo con la exactitud promedio (*mean accuracy*).

Así que calculemos el desempeño con los sets de entrenamiento y validación:

In [23]:
print(f'Exactitud promedio entrenamiento: {bosque.score(x_train,y_train)}')
print(f'Exactitud promedio validación: {bosque.score(x_val, y_val)}')

Exactitud promedio entrenamiento: 1.0
Exactitud promedio validación: 0.5


En este caso vemos que el modelo tiene *overfitting*, pues alcanza un 100% de exactitud con el set de entrenamiento y tan sólo un 50% con el set de prueba.

En realidad es de esperar pues tenemos poquísimos datos y no hemos modificado ningún parámetro por defecto del modelo.

En una situación real deberíamos recolectar más datos y re-entrenar el modelo, posiblemente afinando sus hiperparámetros (pero esto será tema de un tutorial más avanzado).

El cuarto y último paso es poner a prueba el modelo. Para ello podemos primero ver el *score* con el set de prueba:

In [24]:
bosque.score(x_test,y_test)

0.5

Que sigue siendo del 50% por los mismos motivos mencionados anteriormente.

Y, suponiendo que estamos conformes con estos resultados, lo que faltaría sería generar predicciones usando el método `predict()`.

Tomemos nuevamente el set de prueba y generemos predicciones con el modelo entrenado:

In [25]:
y_pred = bosque.predict(x_test)

Y como tenemos tan pocos datos podemos imprimir el comparativo entre las categorías reales (almacenadas en `y_test`) y las categorías predichas (almacenadas en `y_pred`):

In [26]:
print('Categorías reales:   ', y_test)
print('Categorías predichas:', y_pred)

Categorías reales:    [0 1 0 0]
Categorías predichas: [1 0 0 0]


Y vemos que en efecto de los 4 datos sólo dos (los dos últimos) son clasificados correctamente.

### 5. *Pipelines* (tuberías????)

Es posible combinar transformadores y estimadores en un sólo objeto: una *pipeline*.

Una *pipeline* nos permite hacer lo mismo que con los bloques separados, pero tiene ciertas ventajas:

1. El código es más compacto
2. Evita lo que se conoce como la fuga de datos (*data leakage*): que los datos de validación sean "vistos" por el modelo cuando hacemos el entrenamiento

Para hacer un comparativo veamos primero cómo sería el flujo completo de trabajo sin *pipelines*:

In [29]:
# Flujo de trabajo sin "pipelines"
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

# Leer datos
X = np.load('Data/particiones-datos-balanceados.npz')['X']
Y = np.load('Data/particiones-datos-balanceados.npz')['Y']

# Partición en entrenamiento, validación y prueba
x_train, x_resto, y_train, y_resto = train_test_split(
    X, Y, test_size=0.4, random_state=123
)
x_val, x_test, y_val, y_test = train_test_split(
    x_resto, y_resto, test_size=0.5, random_state=321
)

# Escalamiento
scaler = MinMaxScaler(feature_range=(-1,1))
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)
x_test_s = scaler.transform(x_test)

# Creación, entrenamiento y validación del modelo
bosque = RandomForestClassifier()
bosque.fit(x_train, y_train)
print(bosque.score(x_test,y_test))

# Generación de predicciones
print(bosque.predict(x_test))

0.5
[0 1 1 0]


In [28]:
# Con pipelines

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

# Leer datos
X = np.load('Data/particiones-datos-balanceados.npz')['X']
Y = np.load('Data/particiones-datos-balanceados.npz')['Y']

# Partición en entrenamiento, validación y prueba
x_train, x_resto, y_train, y_resto = train_test_split(
    X, Y, test_size=0.4, random_state=123
)
x_val, x_test, y_val, y_test = train_test_split(
    x_resto, y_resto, test_size=0.5, random_state=321
)

#------- PIPELINE: SE INTERCONECTAN PREPROCESAMIENTO Y MODELO -------

# Instanciar la pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler(feature_range=(-1,1))),
    ('classifier', RandomForestClassifier())
])

# Entrenar la pipeline
pipeline.fit(x_train, y_train)

# Evaluar la pipeline
print(pipeline.score(x_test, y_test))

# Y generar predicciones
print(pipeline.predict(x_test))

0.25
[0 1 0 0]
